In [1]:
from pymol import cmd
import py3Dmol

from vina import Vina

import pandas as pd
import numpy as np

from openbabel import pybel

from rdkit import Chem
from rdkit.Chem import AllChem,rdFMCS, Draw

from meeko import MoleculePreparation
from meeko import obutils

import MDAnalysis as mda
from MDAnalysis.coordinates import PDB

import prolif as plf
from prolif.plotting.network import LigNetwork


import sys, os, random
sys.path.insert(1, 'utilities/')

from multiprocessing import Pool

from utils import fix_protein, getbox, generate_ledock_file, dok_to_sdf

import warnings
warnings.filterwarnings("ignore")

%config Completer.use_jedi = False

In [2]:
os.chdir('test/Reverse_Docking/')

In [3]:
targets=['3LM5','2O3P','6M89','6QCD']

In [4]:
for pdb_code in targets:
    cmd.fetch(code=pdb_code,type='pdb1')
    cmd.select(name='Prot',selection='polymer.protein')
    cmd.save(filename='clean_targets/'+pdb_code+'_clean.pdb',format='pdb',selection='Prot')
    cmd.delete('all')
    
    fix_protein(filename='clean_targets/'+pdb_code+'_clean.pdb',addHs_pH=7.4,try_renumberResidues=True,output='clean_targets/'+pdb_code+'_clean_H.pdb')

 PyMOL not running, entering library mode (experimental)


In [5]:
ligand='c1cc(c(cc1C2=C(C(=O)c3c(cc(cc3O2)O)O)O)O)O'

In [6]:
mol=pybel.readstring(string=ligand,format='smiles')
mol.make3D()
out=pybel.Outputfile(filename='Quercetin.mol2',format='mol2',overwrite=True)
out.write(mol)
out.close()

In [11]:
for clean_pdb in os.listdir('clean_targets/'):
    if 'H' in clean_pdb:
        !../../bin/fpocket -f {'clean_targets/'+clean_pdb} -d > {'targets_pockets/'+clean_pdb.split('_')[0] +'.csv'}